In [137]:
from typing import Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F


In [160]:
class RouterBase(nn.Module):
    """
    Base class for routers.
    """

    def __init__(
        self,
        embed_dim: int,
        num_experts: int,
        expert_capacity: int,
        dtype: str = "float32",
        bias: bool = False,
        jitter: float = 0.0,
        num_routable_experts: Optional[int] = None,
    ):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_experts = num_experts
        self.expert_capacity = expert_capacity
        self.dtype = getattr(torch, dtype)
        self.bias = bias
        self.jitter = jitter
        self.classifier = nn.Linear(
            self.embed_dim,
            num_routable_experts
            if num_routable_experts is not None
            else self.num_experts,
            bias=self.bias,
            dtype=self.dtype,
        )

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        raise NotImplementedError

    def _compute_router_probabilities(
        self,
        x: torch.Tensor,
        dim: int = -1,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Computes router probabilities from input hidden states.

        Parameters:
        -----------
        x : torch.Tensor
            Tensor of shape (batch_size, sequence_length, hidden_dim) from which
            router probabilities are computed.

        dim : int, optional
            Dimension along which to compute the softmax. The default is -1, which corresponds
            to token-choice routing. For expert choice routing, this should be -2.

        Returns:
        --------
        router_probabilities : torch.Tensor
            Tensor of shape (batch_size, sequence_length, num_experts) corresponding to
            the probabilities for each token and expert. Used for routing tokens to experts.

        router_logits : torch.Tensor
            Logits tensor of shape (batch_size, sequence_length, num_experts) corresponding
            to raw router logits. This is used for computing router z-loss.
        """
        # float32 is used to ensure stability. See the discussion of "selective precision" in
        # https://arxiv.org/abs/2101.03961.
        # we also store the input dtype so we can cast the output back to the original dtype
        self.input_dtype = x.dtype
        x = x.to(self.dtype)
        if self.jitter > 0:
            x *= torch.empty_like(x).uniform_(1.0 - self.jitter, 1.0 + self.jitter)

        # shape: [batch_size, sequence_length, num_experts]
        logits = self.classifier(x)

        # apply softmax and cast back to the original dtype
        probabilities = F.softmax(logits, dim=dim, dtype=self.dtype).to(
            self.input_dtype
        )
        return probabilities, logits


class TopKRouter(RouterBase):
    """
    This router uses the "token choice of top-k experts" strategy. For example, if k=1, this
    replicates the top-1 routing strategy introduced in the `Switch Transformers`_ paper.
    Alternatively, if k=2, this replicates the top-2 routing strategy introduced in the `GShard`_
    paper. Tokens are routed to their expert of choice until the expert's `expert_capacity` is
    reached. Shared experts, which process all tokens, are implemented as described in the
    `DeepSeqMoE`_ paper.

    .. note::
        There is no guarantee that each token will be processed by an expert,
        or that every expert will receive at least one token.

    If tokens are routed to an expert which is above capacity, they are not processed by any expert
    and their hidden states are passed to the subsequent layer unchanged.


    Parameters:
    -----------
    embed_dim : int
        Embedding dimension.

    num_experts : int
        Number of experts.

    expert_capacity : int
        Maximum number of tokens that can be routed to each expert.

    top_k : int, optional
        Number of top experts to route each token to. The default is 1.

    num_shared_experts : int, optional
        Number of shared experts that process all tokens. The default is 0.

    dtype : str, optional
        Data type to use for router probabilities. The default is "float32".

    bias : bool, optional
        Whether to add bias to the router classifier. The default is ``False``.

    jitter : float, optional
        Amount of jitter to add to the router probabilities. The default is ``0.0``.

    ignore_padding_tokens : bool, optional
        Whether to ignore padding tokens when computing router probabilities.
        The default is ``True``.


    .. _Switch Transformers:
        https://arxiv.org/abs/2101.03961

    .. _GShard:
        https://arxiv.org/abs/2006.16668

    .. _DeepSeqMoE:
        https://arxiv.org/abs/2401.06066
    """

    def __init__(
        self,
        embed_dim: int,
        num_experts: int,
        expert_capacity: int,
        top_k: int = 1,
        num_shared_experts: int = 0,
        dtype: str = "float32",
        bias: bool = False,
        jitter: float = 0.0,
        ignore_padding_tokens: bool = True,
        **kwargs,
    ):
        super().__init__(
            embed_dim=embed_dim,
            num_experts=num_experts,
            expert_capacity=expert_capacity,
            dtype=dtype,
            bias=bias,
            jitter=jitter,
            num_routable_experts=num_experts - num_shared_experts,
        )
        self.top_k = top_k
        self.num_shared_experts = num_shared_experts
        self.ignore_padding_tokens = ignore_padding_tokens

    def forward(
        self, x: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Token choice of top-k experts, with optional shared experts processing all tokens.

        Parameters:
        -----------
        x : torch.Tensor
            Input tensor of shape (batch_size, sequence_length, embed_dim).

        Returns:
        --------
        expert_mask : torch.Tensor
            Binary mask tensor of shape (batch_size, sequence_length, num_experts)
            indicating which experts the token should be routed to (including shared experts).

        router_probabilities : torch.Tensor
            Tensor of shape (batch_size, sequence_length, num_experts) containing
            the router probabilities.

        router_logits : torch.Tensor
            Tensor of shape (batch_size, sequence_length, num_experts) containing
            the router logits.
        """
        num_routable_experts = self.num_experts - self.num_shared_experts

        # router
        router_probs, router_logits = self._compute_router_probabilities(x)
        topk_values, topk_indices = torch.topk(router_probs, k=self.top_k, dim=-1)
        expert_mask = F.one_hot(topk_indices, num_classes=num_routable_experts).sum(dim=-2)

        # mask tokens if their desired experts are above capacity
        token_priority = torch.cumsum(expert_mask, dim=-2)
        expert_capacity_mask = token_priority <= self.expert_capacity
        expert_mask = expert_mask * expert_capacity_mask

        # get the probabilities of the top-choice experts for each token
        # router_probs = expert_values * expert_mask

        # Add shared experts processing all tokens
        if self.num_shared_experts > 0:
            shared_expert_mask = torch.ones_like(
                router_probs[..., : self.num_shared_experts]
            )
            expert_mask = torch.cat((shared_expert_mask, expert_mask), dim=-1)

        return expert_mask, router_probs, router_logits


class ExpertChoiceRouter(RouterBase):
    """
    This router uses the "expert choice of top-k tokens" strategy, as originally described
    in the `Mixture-of-Experts with Expert Choice Routing`_ paper. This automatically
    balances the number of tokens processed by each expert, and eliminates the
    need for an auxiliary (load-balancing) router loss.

    .. note::
        There is no guarantee that each token will be processed by an expert. In fact,
        one of the primary benefits of expert choice routing is thought to be their
        ability to heterogeneously devote computation to a subset of highly complex/difficult
        tokens.

    If tokens are not selected by an expert, their hidden states are passed to the
    subsequent layer unchanged.

    Parameters:
    -----------
    embed_dim : int
        Embedding dimension.

    num_experts : int
        Number of experts.

    expert_capacity : int
        Maximum number of tokens that can be routed to each expert.

    num_shared_experts : int, optional
        Number of shared experts that process all tokens. The default is 0.

    dtype : str, optional
        Data type to use for router probabilities. The default is "float32".

    bias : bool, optional
        Whether to add bias to the router classifier. The default is ``False``.

    jitter : float, optional
        Amount of jitter to add to the router probabilities. The default is ``0.0``.

    ignore_padding_tokens : bool, optional
        Whether to ignore padding tokens when computing router probabilities.
        The default is ``True``.

    .. _Mixture-of-Experts with Expert Choice Routing:
        https://arxiv.org/abs/2202.09368
    """

    def __init__(
        self,
        embed_dim: int,
        num_experts: int,
        expert_capacity: int,
        num_shared_experts: int = 0,
        dtype: str = "float32",
        bias: bool = False,
        jitter: float = 0.0,
        ignore_padding_tokens: bool = True,
        **kwargs,
    ):
        super().__init__(
            embed_dim=embed_dim,
            num_experts=num_experts,
            expert_capacity=expert_capacity,
            dtype=dtype,
            bias=bias,
            jitter=jitter,
            num_routable_experts=num_experts - num_shared_experts,
        )
        self.num_shared_experts = num_shared_experts
        self.ignore_padding_tokens = ignore_padding_tokens

    def forward(
        self, x: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Expert choice of top-k tokens, with optional shared experts that process all tokens.

        Parameters:
        -----------
        x : torch.Tensor
            Input tensor of shape (batch_size, sequence_length, embed_dim).

        Returns:
        --------
        expert_mask : torch.Tensor
            Binary mask tensor of shape (batch_size, sequence_length, num_experts) indicating
            which tokens are selected for each expert and which are processed by shared experts.

        router_probabilities : torch.Tensor
            Tensor of shape (batch_size, sequence_length, num_experts) containing
            the router probabilities.

        router_logits : torch.Tensor
            Tensor of shape (batch_size, sequence_length, num_experts) containing
            the router logits.
        """
        router_probs, router_logits = self._compute_router_probabilities(x, dim=1)
        expert_mask = torch.zeros_like(router_probs)

        # Select top-k tokens for each expert
        for i in range(self.num_experts - self.num_shared_experts):
            _, top_k_indices = torch.topk(
                router_probs[..., i], k=self.expert_capacity, dim=1
            )
            expert_mask[:, :, i].scatter_(1, top_k_indices, 1)

        # Ensure that the mask is binary
        # expert_mask = expert_mask.clamp(max=1)

        # Add shared experts processing all tokens
        if self.num_shared_experts > 0:
            shared_expert_mask = torch.ones_like(
                router_probs[..., : self.num_shared_experts]
            )
            expert_mask = torch.cat((shared_expert_mask, expert_mask), dim=-1)

        return expert_mask, router_probs, router_logits

In [161]:
router = TopKRouter(
    embed_dim=1024, 
    num_experts=16,
    expert_capacity=8, 
    # top_k=2, 
    num_shared_experts=1,
    )


In [162]:
data = torch.randn(10, 100, 1024)



In [163]:
expert_mask, router_probs, router_logits = router(data)

# print(expert_mask.shape)
# print(router_probs.shape)
# print(router_logits.shape)



In [165]:
expert_mask[0][2].shape

torch.Size([16])

In [149]:
router_probs[0][2]

tensor([0.0074, 0.0060, 0.0094, 0.0123, 0.0191, 0.0078, 0.0176, 0.0061, 0.0032,
        0.0068, 0.0101, 0.0066, 0.0323, 0.0079, 0.0119],
       grad_fn=<SelectBackward0>)

In [150]:
router_logits[0][2]

tensor([-0.1264, -0.5073, -0.0097,  0.4879,  0.7862, -0.1197,  0.8284, -0.3132,
        -0.9528, -0.3334,  0.2162, -0.2432,  1.4168, -0.1742,  0.3891],
       grad_fn=<SelectBackward0>)

In [151]:
expert_mask.shape

torch.Size([10, 100, 16])

In [152]:
expert_mask.sum(dim=-1)



tensor([[2., 2., 2., 3., 2., 2., 2., 4., 3., 1., 5., 2., 2., 3., 3., 4., 3., 2.,
         2., 1., 3., 2., 4., 2., 1., 2., 2., 2., 2., 3., 1., 3., 2., 1., 2., 4.,
         4., 3., 2., 2., 3., 1., 4., 2., 3., 3., 3., 1., 1., 2., 3., 3., 2., 1.,
         2., 2., 3., 1., 4., 4., 1., 3., 2., 2., 2., 2., 1., 1., 2., 1., 3., 3.,
         2., 2., 3., 2., 3., 3., 2., 1., 1., 2., 2., 2., 2., 2., 1., 1., 1., 2.,
         2., 1., 3., 1., 3., 1., 2., 1., 1., 3.],
        [1., 3., 2., 4., 2., 2., 3., 1., 3., 2., 3., 4., 3., 3., 1., 1., 3., 1.,
         2., 3., 1., 1., 2., 4., 2., 3., 1., 1., 3., 2., 3., 3., 3., 4., 2., 1.,
         1., 6., 2., 3., 3., 2., 1., 1., 1., 1., 3., 3., 3., 3., 1., 3., 4., 1.,
         3., 3., 1., 4., 2., 2., 3., 2., 2., 1., 1., 2., 1., 3., 3., 1., 2., 1.,
         2., 3., 1., 1., 2., 4., 2., 2., 2., 2., 1., 1., 2., 2., 1., 2., 1., 2.,
         3., 3., 3., 2., 3., 3., 2., 3., 1., 2.],
        [1., 4., 2., 2., 1., 1., 2., 2., 2., 3., 3., 2., 2., 2., 2., 1., 3., 2.,
         

In [30]:
indices = torch.tensor([2, 6], dtype=torch.long)
values = torch.tensor([1, 2], dtype=torch.float32)

x = torch.zeros(10)
x[indices] = values
print(x)



tensor([0., 0., 1., 0., 0., 0., 2., 0., 0., 0.])
